# Petite introduction au TAL avec nltk (et autres)

## Stockage des XML
#### On va commencer par stocker nos xml dans des String, hugoIntegral, stendhalIntegral, flaubertIntegral

In [1]:
from lxml import etree
treeHugo = etree.parse("hugo_miserables.xml")
treeStendhal = etree.parse("stendhal_lerougeetlenoir.xml")
treeFlaubert=etree.parse("flaubert_bovary.xml")
listeHugo=list()
listeStendhal=list()
listeFlaubert=list()
listeHugo.extend(user.text for user in treeHugo.xpath("/TEI/text/body/div/div/div/p"))
listeFlaubert.extend(user.text for user in treeFlaubert.xpath("/TEI/text/body/div/div/div/p"))
listeStendhal.extend(user.text for user in treeStendhal.xpath("/TEI/text/body/div/div/div/p"))
hugoIntegral=''
stendhalIntegral=''
flaubertIntegral=''
for e in listeHugo:
    if (isinstance(e,str)):
        hugoIntegral+=e+' '
for e in listeStendhal:
    if (isinstance(e,str)):
        stendhalIntegral+=e+' '
for e in listeFlaubert:
    if (isinstance(e,str)):
        flaubertIntegral+=e+' '

## Tokénization et nettoyage
#### Ici on utilise un word-tokenizer basique, basé sur le Penn Bank

In [2]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
#nltk.download()
# chargement du tokenizer
#tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
strTest='Je suis Marianne et je fais un simple test'
tokenizer=TreebankWordTokenizer()
#tokenizer = nltk.data.load('./nltk_data/tokenizers/punkt/french.pickle')
tokens = tokenizer.tokenize(strTest)
print (tokens)

['Je', 'suis', 'Marianne', 'et', 'je', 'fais', 'un', 'simple', 'test']


#### Ici, on va supprimer les stopwords qui faussent souvent les données

In [18]:
from nltk.corpus import stopwords
#stop_words=set(stopwords.words("french"))
with open('stopwords.txt', encoding='utf-8') as f:
    stop_words = f.read().split('\n')
#texte_sans_SW=[]
#for t in tokens :
#    if t not in stop_words :
#        texte_sans_SW.append(t)
texte_sans_SW=[t for t in tokens if not t in stop_words]
print (texte_sans_SW)

['Je', 'Marianne', 'simple', 'test']


## Racinisation, lemmatisation et étiquetage
#### Souvent, pour l'analyse, on utilise un outil léger, un stemmer qui réduit chaque mot à sa racine

In [4]:
from nltk.stem.snowball import FrenchStemmer
st=FrenchStemmer()
stemmed_text=[st.stem(mot) for mot in texte_sans_SW]
print (stemmed_text)

['je', 'suis', 'mariann', 'et', 'je', 'fais', 'un', 'simpl', 'test']


In [5]:
stringATagger=''
for tok in texte_sans_SW:
    stringATagger+=tok+' '

#### Mais souvent, il est plus pertinent de lemmatiser et de postagger un texte. La lemmatisation ici est faite avec un wrapper (un adaptateur) du lemmatiseur TreeTagger, et l'étiquetage syntaxique léger avec le Stanford POStagger.

Attention, vous aurez besoin ici d'un wrapper, que vous obtiendrez en exécutant cette commande :
    pip install treetaggerwrapper

In [6]:
import treetaggerwrapper
import re
#1) build a TreeTagger wrapper:
tagger = treetaggerwrapper.TreeTagger(TAGDIR='/home/odysseus/Bureau/PythonCourse/treetagger/',TAGPARFILE='/home/odysseus/Bureau/PythonCourse/treetagger/models/french.par')
#2) tag your text.
tags = tagger.tag_text(stringATagger)
lems=list()
for wordTagged in tags:
    lems.append(re.split(r'\t+', wordTagged)[2])
print(lems)

['je', 'suivre|être', 'Marianne', 'et', 'je', 'faire', 'un', 'simple', 'test']


In [7]:
from nltk.tag import StanfordPOSTagger
jar = 'stanford/stanford-postagger-3.7.0.jar'
model = 'models/french.tagger'
pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
tags=pos_tagger.tag(stringATagger.split())
print (tags)

[('Je', 'CLS'), ('suis', 'V'), ('Marianne', 'NPP'), ('et', 'CC'), ('je', 'CLS'), ('fais', 'V'), ('un', 'DET'), ('simple', 'ADJ'), ('test', 'NC')]


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Similarité de documents
#### Ici ça va vous sembler répétitif, mais on va refaire le même procédé (sans les prints) sur nos trois xml, pour voir, entre autre, la proximité des documents.

In [9]:
synopses=list()
synopses.append(hugoIntegral)
synopses.append(flaubertIntegral)
synopses.append(stendhalIntegral)

#### Une des principales mesures de proximité entre documents, et qui exige une bonne tokenization, voire lemmatisation, est la TF-IDF (Term Frequency-Inverse Document Frequency). Le principe de la TF-IDF est de transformer les documents en des vecteurs dont le poids des mots est déterminé en fonction de leur fréquence.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
def tokenize_my_way(text):
    filtered_tokens = []
    tokens = tokenizer.tokenize(text)
    for token in tokens:
        if re.search('[a-zA-ZéèêàâîôûçïöüäëÉ]', token):
            filtered_tokens.append(token)
    return filtered_tokens


In [12]:
stops=list()
for sw in stop_words:
    stops.append(sw)
#tfidf_vectorizer = TfidfVectorizer(max_df=1, max_features=200000,
                                 #min_df=0.0, stop_words=stops,
                                 #use_idf=True, tokenizer=tokenize_my_way, ngram_range=(1,3))

tfidf_matrix = TfidfVectorizer().fit_transform(synopses)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)